In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
import b3d.chisight.shared.particle_system as ps
import genjax
from genjax import Pytree
import jax
from b3d import Pose
import b3d

In [29]:
genjax.pretty()

In [30]:
key = jax.random.PRNGKey(125)

num_timesteps = Pytree.const(4)
num_particles = Pytree.const(5)
num_clusters = Pytree.const(3)
relative_particle_poses_prior_params = (Pose.identity(), .5, 0.25)
initial_object_poses_prior_params = (Pose.identity(), 2., 0.5)
camera_pose_prior_params = (Pose.identity(), 0.1, 0.1)
instrinsics = Pytree.const(b3d.camera.Intrinsics(120, 100, 50., 50., 50., 50., 0.001, 16.))
sigma_obs = 0.2

trace = ps.sparse_gps_model.simulate(key, (
    (
        num_timesteps, # const object
        num_particles, # const object
        num_clusters, # const object
        relative_particle_poses_prior_params,
        initial_object_poses_prior_params,
        camera_pose_prior_params
    ),
    (instrinsics, sigma_obs)
))

TypeError: Cannot concatenate arrays with shapes that differ in dimensions other than the one being concatenated: concatenating along dimension 0 for shapes (1, 5), (3, 3).

In [31]:
tr = ps.initial_particle_system_state.simulate(key, (
    num_particles, num_clusters, relative_particle_poses_prior_params, initial_object_poses_prior_params, camera_pose_prior_params
))

In [33]:
tr.get_choices()

XorChm(...)

In [36]:
(state0, init_retval) = tr.get_retval()
init_retval

{'absolute_particle_poses': Pose(position=Array([[ 0.40394464,  0.49178421,  0.83125967],
        [ 1.1557399 ,  0.1073809 , -1.0004435 ],
        [ 0.7466202 , -0.11125076, -1.0334555 ],
        [ 0.42588845,  0.15040515,  0.7240628 ],
        [-0.08075588,  0.03178127,  0.9834795 ]], dtype=float32), quaternion=Array([[ 0.31261247,  0.3264106 ,  0.45340466,  0.7682147 ],
        [ 0.04813731, -0.29024395, -0.20838895,  0.93274605],
        [ 0.12582801, -0.71169275, -0.09951743,  0.6839277 ],
        [-0.09185741,  0.30670545,  0.40737277,  0.855302  ],
        [ 0.03260923,  0.1007082 ,  0.2827858 ,  0.9533241 ]],      dtype=float32)),
 'camera_pose': Pose(position=Array([-0.00054856,  0.0540217 , -0.05834808], dtype=float32), quaternion=Array([ 0.06375018, -0.02930492, -0.1331465 ,  0.98860973], dtype=float32)),
 'object_poses': Pose(position=Array([[-0.8351105 ,  1.3851395 ,  0.63394755],
        [ 0.9989229 , -0.10820086, -1.0928634 ],
        [-0.02473093,  0.27078107,  0.85259247]], dtype=float32), quaternion=Array([[ 0.05015931, -0.3799355 ,  0.16383284,  0.909006  ],
        [ 0.03074602, -0.5201218 ,  0.08101467,  0.8496851 ],
        [-0.00533034,  0.2970283 ,  0.3590314 ,  0.88478374]],      dtype=float32)),
 'relative_particle_poses': Pose(position=Array([[ 0.39352402, -0.1140423 ,  0.25532994],
        [ 0.17484564,  0.18277925, -0.12501697],
        [-0.05999853,  0.03789828,  0.24931735],
        [ 0.2468853 , -0.40325543,  0.10247313],
        [-0.25207934, -0.11491695,  0.02532204]], dtype=float32), quaternion=Array([[ 0.2632069 , -0.05403322,  0.21994649,  0.9377772 ],
        [-0.11967833,  0.22821863, -0.2687445 ,  0.92810005],
        [-0.02353261, -0.26224265, -0.18353081,  0.94709635],
        [-0.08759926,  0.04812734,  0.02770715,  0.9946067 ],
        [-0.01390432, -0.2072743 , -0.0818463 ,  0.9747539 ]],      dtype=float32)),
 'vis_mask': <jax.Array([0, 0, 1, 0, 0], dtype=int32)>}

In [37]:
scantr = ps.particle_system_state_step.scan(n=(num_timesteps.const - 1)).simulate(
    key, (state0, None)
)

In [38]:
scantr.get_choices()

IdxChm(...)

In [39]:
(final_state, scan_retval) = scantr.get_retval()
scan_retval

{'absolute_particle_poses': Pose(position=Array([[[ 0.36036593,  0.4931063 ,  0.87947124],
         [ 1.1646038 ,  0.05811405, -1.1079048 ],
         [ 0.73829836, -0.12721162, -1.094248  ],
         [ 0.38899153,  0.15259093,  0.77110964],
         [-0.15376458,  0.04543683,  0.94996977]],
 
        [[ 0.37076294,  0.3681712 ,  0.90084624],
         [ 1.196027  ,  0.07400203, -1.0740566 ],
         [ 0.77749217, -0.12785229, -1.0928155 ],
         [ 0.3501452 ,  0.05677184,  0.72444403],
         [-0.16809943, -0.05037972,  0.9652722 ]],
 
        [[ 0.38736004,  0.3257344 ,  0.9398421 ],
         [ 1.2504895 ,  0.02589861, -1.0647875 ],
         [ 0.8366114 , -0.15606686, -1.1737005 ],
         [ 0.29607272,  0.02570063,  0.7661794 ],
         [-0.24314705,  0.06024784,  0.9809019 ]]], dtype=float32), quaternion=Array([[[ 0.28782642,  0.2602054 ,  0.46717957,  0.7944762 ],
         [-0.04225804, -0.291873  , -0.22387134,  0.9289274 ],
         [ 0.03933909, -0.7102853 , -0.07273035,  0.6990405 ],
         [-0.11164563,  0.23999774,  0.38960102,  0.8821267 ],
         [ 0.0196385 ,  0.02642748,  0.27185038,  0.96177614]],
 
        [[ 0.41381675,  0.26070198,  0.43473133,  0.75617373],
         [ 0.06674349, -0.25198773, -0.23636203,  0.93604517],
         [ 0.13210969, -0.6864102 , -0.13979195,  0.70131755],
         [ 0.0200845 ,  0.2717025 ,  0.3836268 ,  0.8823859 ],
         [ 0.14041534,  0.07309642,  0.23153813,  0.9598597 ]],
 
        [[ 0.42608923,  0.19002779,  0.34147373,  0.81592476],
         [ 0.09314049, -0.3472136 , -0.25124246,  0.8986906 ],
         [ 0.13637125, -0.75708413, -0.14683306,  0.6218252 ],
         [ 0.03734541,  0.2480718 ,  0.26456118,  0.9311676 ],
         [ 0.13413665,  0.03236134,  0.10862075,  0.9844601 ]]],      dtype=float32)),
 'camera_pose': Pose(position=Array([[-0.05851199,  0.08131988, -0.12015802],
        [-0.03376154,  0.10507603, -0.20219323],
        [-0.02877734,  0.0587935 , -0.276977  ]], dtype=float32), quaternion=Array([[ 0.10911258,  0.18976991, -0.02298202,  0.9754762 ],
        [ 0.08588515,  0.0251347 , -0.07712492,  0.9929974 ],
        [ 0.20970307,  0.17755964,  0.11609443,  0.95447326]],      dtype=float32)),
 'object_poses': Pose(position=Array([[[-0.87120515,  1.3736898 ,  0.6607467 ],
         [ 0.987428  , -0.15515342, -1.1601944 ],
         [-0.07241641,  0.28432354,  0.832894  ]],
 
        [[-0.88771987,  1.4047782 ,  0.7261908 ],
         [ 1.0251832 , -0.12936762, -1.1692494 ],
         [-0.08224714,  0.2019974 ,  0.88573414]],
 
        [[-0.9435097 ,  1.4065702 ,  0.74644756],
         [ 1.0944561 , -0.1681515 , -1.1974776 ],
         [-0.0918551 ,  0.28034472,  0.9030608 ]]], dtype=float32), quaternion=Array([[[ 0.02159333, -0.4294057 ,  0.25887853,  0.86494297],
         [-0.05757808, -0.49832183,  0.08644408,  0.8607483 ],
         [-0.02166905,  0.22728433,  0.34740812,  0.90949434]],
 
        [[-0.00793305, -0.43463644,  0.34698018,  0.8310433 ],
         [ 0.05230644, -0.49371713,  0.04711475,  0.8667686 ],
         [ 0.10820736,  0.26193216,  0.33234182,  0.8995731 ]],
 
        [[ 0.08902398, -0.46995282,  0.35609448,  0.8027551 ],
         [ 0.04334753, -0.5824462 ,  0.02863756,  0.8112074 ],
         [ 0.12457285,  0.22612935,  0.21380606,  0.94214344]]],      dtype=float32)),
 'relative_particle_poses': Pose(position=Array([[[ 0.39352402, -0.1140423 ,  0.25532994],
         [ 0.17484564,  0.18277925, -0.12501697],
         [-0.05999853,  0.03789828,  0.24931735],
         [ 0.2468853 , -0.40325543,  0.10247313],
         [-0.25207934, -0.11491695,  0.02532204]],
 
        [[ 0.39352402, -0.1140423 ,  0.25532994],
         [ 0.17484564,  0.18277925, -0.12501697],
         [-0.05999853,  0.03789828,  0.24931735],
         [ 0.2468853 , -0.40325543,  0.10247313],
         [-0.25207934, -0.11491695,  0.02532204]],
 
        [[ 0.39352402, -0.1140423 ,  0.25532994],
         [ 0.17484564,  0.18277925, -0.12501697],
         [-0.05999853,  0.0378

In [40]:
for key in scan_retval.keys():
    print(key, scan_retval[key].shape, init_retval[key].shape)

absolute_particle_poses (3, 5) (5,)
camera_pose (3,) ()
object_poses (3, 3) (3,)
relative_particle_poses (3, 5) (5,)
vis_mask (3, 3) (5,)


In [ ]:
trace

<unknown>:23: SyntaxWarning: invalid escape sequence '\p'


StaticTrace(...)

In [43]:
from b3d.chisight.sparse.pose_utils import uniform_pose_in_ball
posetr = uniform_pose_in_ball.simulate(key, (Pose.identity(), 0.1, 0.2))

TypeError: Method[1m[35m genjax._src.generative_functions.distributions.distribution.Distribution.simulate()[0m parameter [1m[34mkey[0m=[1m[31m'vis_mask'[0m violates type hint [1m[32mtyping.Union[jaxtyping.Key[Array, ''], jaxtyping.UInt32[Array, '2']][0m, as [1m[33mstr [0m[1m[31m'vis_mask'[0m not [1m[32m<class "jaxtyping.Key[Array, '']"> or <class "jaxtyping.UInt32[Array, '2']">[0m.

In [11]:
trace.get_choices()

EmptyChm(...)